In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,24796
2,Huelva,Confirmados PDIA 14 días,267
3,Huelva,Tasa PDIA 14 días,"52,02954186721749"
4,Huelva,Confirmados PDIA 7 días,131
5,Huelva,Total Confirmados,25004
6,Huelva,Curados,17910
7,Huelva,Fallecidos,339


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  24796.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6588.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 6849 personas en los últimos 7 días 

Un positivo PCR cada 2935 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,24796.0,131.0,267.0,513170.0,25.527603,52.029542,25.0
Huelva-Costa,15120.0,74.0,161.0,289548.0,25.557075,55.603907,12.0
Condado-Campiña,6963.0,54.0,101.0,156231.0,34.564203,64.647861,11.0
Ayamonte,1223.0,19.0,34.0,21104.0,90.030326,161.106899,4.0
Cartaya,960.0,6.0,20.0,20083.0,29.876015,99.586715,3.0
Huelva (capital),6588.0,21.0,49.0,143837.0,14.599860,34.066339,2.0
Lucena del Puerto,162.0,5.0,11.0,3261.0,153.327200,337.319841,2.0
Moguer,837.0,7.0,11.0,21867.0,32.011707,50.304111,2.0
Palma del Condado (La),983.0,5.0,10.0,10801.0,46.292010,92.584020,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Puebla de Guzmán,102.0,7.0,20.0,3092.0,226.390686,646.830530,0.0
Lucena del Puerto,162.0,5.0,11.0,3261.0,153.327200,337.319841,2.0
San Silvestre de Guzmán,13.0,2.0,2.0,614.0,325.732899,325.732899,0.0
San Bartolomé de la Torre,204.0,5.0,11.0,3761.0,132.943366,292.475405,1.0
Corteconcepción,24.0,1.0,1.0,536.0,186.567164,186.567164,NaN
Ayamonte,1223.0,19.0,34.0,21104.0,90.030326,161.106899,4.0
Villablanca,177.0,1.0,3.0,2885.0,34.662045,103.986135,0.0
Cartaya,960.0,6.0,20.0,20083.0,29.876015,99.586715,3.0
Almonte,878.0,10.0,24.0,24507.0,40.804668,97.931203,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cartaya,960.0,6.0,20.0,20083.0,29.876015,99.586715,3.0,0.300000
Alosno,123.0,1.0,3.0,3933.0,25.425884,76.277651,1.0,0.333333
Villablanca,177.0,1.0,3.0,2885.0,34.662045,103.986135,0.0,0.333333
Puebla de Guzmán,102.0,7.0,20.0,3092.0,226.390686,646.830530,0.0,0.350000
Almonte,878.0,10.0,24.0,24507.0,40.804668,97.931203,0.0,0.416667
Huelva (capital),6588.0,21.0,49.0,143837.0,14.599860,34.066339,2.0,0.428571
San Bartolomé de la Torre,204.0,5.0,11.0,3761.0,132.943366,292.475405,1.0,0.454545
Bollullos Par del Condado,621.0,5.0,11.0,14387.0,34.753597,76.457913,1.0,0.454545
Lucena del Puerto,162.0,5.0,11.0,3261.0,153.327200,337.319841,2.0,0.454545
